In [ ]:
# init repo notebook
!git clone https://github.com/rramosp/ppdl.git > /dev/null 2> /dev/null
!mv -n ppdl/content/init.py ppdl/content/local . 2> /dev/null
!pip install -r ppdl/content/requirements.txt > /dev/null

# Bayesian Linear Regression
---

In this notebook we'll implement a Bayesian linear regression model and review its mathematical details, first, let us import some base libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.optimizers import Adam
from IPython.display import display, Math
plt.style.use("ggplot")

In [ ]:
tfd = tfp.distributions

## Probabilistic Linear Regression
---

The linear regression is one of the simplest models in machine learning, the main goal is to compute an estimation $\tilde{\mathbf{y}} \in \mathbb{R} ^ N$ (where $N$ is the number of samples) of a target variable $\mathbf{y} \in \mathbb{R} ^ N$ using some input variables $\mathbf{X} \in \mathbb{R} ^ {N \times m}$ (where $m$ is the number of variables) through a linear model:

$$
\tilde{\mathbf{y}} =  \mathbf{x} \cdot \mathbf{w}
$$

Conventionally, the parameters $\mathbf{w} \in \mathbb{R} ^ m$ are estimated through the minimization of the Mean Squared Error (MSE). However, this is equivalent to the Maximum Likelihood Estimation (MLE) of a probabilistic model. More precisely, we assume that the distribution of the label's noise is given by a normal distribution, so it is possible to define the following model:

$$
\begin{split}
\mathbf{y} &= \mathbf{\tilde{y}} + \epsilon\\
\mathbf{y} &= \mathbf{X} \cdot \mathbf{w} + \epsilon
\end{split}
$$

Where $\epsilon \sim \mathcal{N}(\mu = 0, \sigma=E)$ and $E$ is the noise's magnitude or standard error. This can be summarized as the distribution of the target variables as follows:

$$
P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E) = \mathcal{N}(\mathbf{y} = \mathbf{X} \cdot \mathbf{w}, \sigma=E)
$$

The MLE would then consist on determining $\mathbf{w}$ and $E$ such that the likelihood probability is maximized, or equivalently, minimizing the negative log-likelihood.

Let's see an example over some synthetic data:

In [ ]:
x = tf.random.uniform(
        shape = (1000, 1),
        minval = -1,
        maxval = 1
        )
X = tf.concat([x, tf.ones_like(x)], axis=1)
display(Math(r"\mathbf{X}:"))
display(X[:5])
display(X.shape)

Let's assume the following values for $\mathbf{w}$ and $E$:

In [ ]:
w_real = tf.constant([[1.0], [-1.0]])
e_real = tf.constant(0.1)
display(Math(r"\mathbf{w}:"))
display(w_real)
display(Math(r"E:"))
display(e_real)

Using these theoretical values, We can generate the $\mathbf{y}$ values:

In [ ]:
y = X @ w_real + tf.random.normal(shape=(1000, 1), mean=0, stddev=e_real)
display(Math(r"\mathbf{y}:"))
display(y[:5])
display(y.shape)

We can visualize the generated dataset:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(tf.squeeze(x), tf.squeeze(y), alpha=0.2)
ax.set_xlabel(r"$\mathbf{x}$")
ax.set_ylabel(r"$\mathbf{y}$")
fig.show()

For the estimation of these values We'll use the automatic differentiation of `tensorflow`, to this end, we must define an initial set of parameters:

In [ ]:
w = tf.Variable([[0.0], [0.0]])
e = tf.Variable(1.0)

Now, We can define a `tensorflow_probability` distribution:

In [ ]:
dist = tfd.Normal(loc=X @ w, scale=e)
display(dist)

Let's define the training loop:

In [ ]:
n_iters = 100
optimizer = Adam(learning_rate=1e-2)
training_variables = [w, e]
for i in range(n_iters):
    with tf.GradientTape() as t:
        dist = tfd.Normal(loc=X @ w, scale=e)
        nll = -tf.reduce_sum(dist.log_prob(y))
        grads = t.gradient(nll, training_variables)
        optimizer.apply_gradients(zip(grads, training_variables))

We can compare the real parameters $\mathbf{w}$ and the estimated ones $\tilde{\mathbf{w}}$

In [ ]:
display(Math(r"\mathbf{w}"))
display(w_real)
display(Math(r"\tilde{\mathbf{w}}"))
display(w)

Also the noise's magnitude $E$ and the estimated one $\tilde{E}$

In [ ]:
display(Math(r"E"))
display(e_real)
display(Math(r"\tilde{E}"))
display(e)

As you can see, it's a valid probabilistic model. We can generate predictions from it:

In [ ]:
x_test = tf.cast(
        tf.reshape(tf.linspace(start=-1, stop=1, num=100), (-1, 1)),
        tf.float32
        )
X_test = tf.concat([x_test, tf.ones_like(x_test)], axis=1)
y_pred = X_test @ w
y_pred_high = y_pred + 3 * tf.ones_like(y_pred) * e
y_pred_low = y_pred - 3 * tf.ones_like(y_pred) * e

Let's visualize the predictions:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(tf.squeeze(x), tf.squeeze(y), alpha=0.2, label="data")
ax.plot(tf.squeeze(x_test), tf.squeeze(y_pred), label=r"$\tilde{y}$", color="k")
ax.fill_between(
    tf.squeeze(x_test),
    tf.squeeze(y_pred_low),
    tf.squeeze(y_pred_high),
    alpha=0.5, color="k"
)
ax.set_xlabel(r"$\mathbf{x}$")
ax.set_ylabel(r"$\mathbf{y}$")
ax.legend()
fig.show()

## Bayesian Approach
---

One of the most important aspects of the probabilistic approach for linear regression is the ability to incorporate prior information into the model through a Bayesian approach.

For instance, we can assume that the model's $\mathbf{w}$ are very likely to be within a circle of radius 3, as shown in the following figure:

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
circle = plt.Circle((0, 0), 3, color="#33333355", label="Prior")
ax.add_patch(circle)
ax.scatter([w_real[0]], [w_real[1]], label=r"$\mathbf{w}$")
ax.set_xlabel(r"$w_0$")
ax.set_ylabel(r"$w_1$")
ax.legend()
fig.show()

In fact, this is equivalent to the $L_2$ regularization that is typically used in models like Ridge Regression or neural networks. Similarly, this behavior can also be represented through a circular distribution, more precisely:

$$
P(\mathbf{w}) = \mathcal{N}(\mathbf{w} = [0, 0], \Sigma=\mathbf{I})
$$

It is well known that around 96% of the density for this distribution is within a circle with radius 3 and centered at $[0, 0]$. This can be seen in the following figure:

In [ ]:
x_range = np.linspace(-5, 5, 100)
X1, X2 = np.meshgrid(x_range, x_range)
X_grid = np.concatenate([X1.reshape(-1, 1), X2.reshape(-1, 1)], axis=1)
dist = tfd.MultivariateNormalDiag(loc=[0., 0.], scale_diag=[1., 1.])
probs = dist.prob(X_grid).numpy().reshape(X1.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.contourf(X1, X2, probs, cmap="gray", label=r"$P(\mathbf{w})$")
ax.scatter([w_real[0]], [w_real[1]], label=r"$\mathbf{w}$")
ax.set_xlabel(r"$w_0$")
ax.set_ylabel(r"$w_1$")
ax.legend()
fig.show()

We can use this information as a prior distribution in a Bayesian approach, let's assume that the standard error $E$ is constant, therefore:

$$
\begin{split}
P(\mathbf{w}) = \mathcal{N}(\mathbf{w}=[0, 0], \Sigma=\mathbf{I}) \\
P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E) = \mathcal{N}(\mathbf{y} = \mathbf{X} \cdot \mathbf{w}, \sigma=E)
\end{split}
$$

Using the Bayes rule, we obtain:

$$
P(\mathbf{w} | \mathbf{X}, \mathbf{y}, E) = \frac{P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E) P(\mathbf{w})}{P(\mathbf{X}, \mathbf{y}, E)}
$$

## Maximum Aposteriori Estimation
---

The maximum aposteriori estimation (MAP) is similar to maximum likelihood estimation (MLE), however, in this case we perform the following optimization:

$$
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} \left( P(\mathbf{w} | \mathbf{X}, \mathbf{y}, E) \right)
$$

Which is equivalent of the optimiziation of the log-posterior considering the convexity of the log function:

$$
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} \left(\log{P(\mathbf{w} | \mathbf{X}, \mathbf{y}, E)}\right)
$$

We can use the Bayes rule:

$$
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} \left( \log{P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E) } + \log{P(\mathbf{w})} - \log{P(\mathbf{X}, \mathbf{y}, E)} \right)
$$

However, the term $\log{P(\mathbf{X}, \mathbf{y}, E)}$ does not depend on $\mathbf{w}$ and the optimization can be simplified to:

$$
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} \left( \log{P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E)} + \log{P(\mathbf{w})} \right)
$$

### Closed-form Solution
---

We can obtain an analytical solution in some cases, for example, when both the prior and the posterior are normal.

Let's see that case:

$$
\begin{split}
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} \left( \log{P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E)} + \log{P(\mathbf{w})} \right)\\
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} \left( \log{\mathcal{N} (y = \mathbf{X} \cdot \mathbf{w}, \sigma=E)} + \log{\mathcal{N}(\mathbf{w} = [0, 0], \sigma=\tau)}\right)\\
\mathbf{w_{map}} = \underset{\mathbf{w}}{\text{argmax}} - \frac{1}{E} (y - \mathbf{X} \cdot \mathbf{w}) ^ 2 - \frac{1}{2\tau ^ 2}||\mathbf{w}|| ^ 2
\end{split}
$$

As you can see, this is equivalent to the optimization of the mean squared error:

$$
\mathcal{L} = (y - \mathbf{X} \cdot \mathbf{w}) ^ 2
$$

Using a regularization term:

$$
\mathcal{R} = ||\mathbf{w}|| ^ 2
$$

And a regularization constant:

$$
\lambda = \frac{1}{2 \tau ^ 2}
$$

We can find a closed-form solution using the derivative:

$$
\frac{\partial}{\partial \mathbf{w}} (\mathcal{L} + \lambda \mathcal{R}) = 0
$$

Which leads to the solution of a Ridge regression model:

$$
\mathbf{w_{map}} = (\mathbf{X} ^ T \cdot \mathbf{X} + \lambda \mathbf{I}) ^ {-1} \mathbf{X} ^ T \mathbf{y}
$$

Let's see this in `tensorflow`:

In [ ]:
l = 0.5
w_map = (
        tf.linalg.pinv(tf.transpose(X) @ X + l * tf.eye(X.shape[1])) @
        tf.transpose(X) @ y
        )
display(w_map)

### Optimization
---

In a more general scenario the distributions may not be normal. However, We can find a MAP estimation using automatic differentiation and `tensorflow_probability`.

Let's see an example with the following likelihood and prior distributions:

$$
\begin{split}
P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E) = N(\mathbf{y} = \mathbf{X} \cdot \mathbf{w}, \sigma=E)\\
P(\mathbf{w}) = \text{Laplace}(\mathbf{w} = [0, 0], \Sigma=\mathbf{I})
\end{split}
$$

We can define this model as a joint distribution:

In [ ]:
model = tfd.JointDistributionNamedAutoBatched({
    "w": tfd.Independent(tfd.Laplace(loc=[0, 0], scale=1), reinterpreted_batch_ndims=1),
    "y": lambda w: tfd.Normal(loc=X @ tf.reshape(w, (-1, 1)), scale=e_real)
    })

And We can find the MAP estimation through a training loop:

In [ ]:
n_iters = 100
w = tf.Variable([0.0, 0.0])
l = 0.5
optimizer = Adam(learning_rate=1e-2)
training_variables = [w]
for i in range(n_iters):
    with tf.GradientTape() as t:
        model = tfd.JointDistributionNamedAutoBatched({
            "w": tfd.Independent(
                tfd.Laplace(loc=[0, 0], scale=l),
                reinterpreted_batch_ndims=1,
            ),
            "y": lambda w: tfd.Normal(loc=X @ tf.reshape(w, (-1, 1)), scale=e_real)
            })
        nll = -tf.reduce_sum(model.log_prob(w = w, y = y))
        grads = t.gradient(nll, training_variables)
        optimizer.apply_gradients(zip(grads, training_variables))

Let's see $\mathbf{w_{map}}$ estimation:

In [ ]:
display(Math(r"\mathbf{w}"))
display(w_real)
display(Math(r"\mathbf{w_{map}}"))
display(w)

As you can see, the weights are lower in comparison to the real ones, which is a result of the regularization.

With this approach, it's possible to optimize the standard error too:

In [ ]:
n_iters = 100
w = tf.Variable([0.0, 0.0])
e = tf.Variable(1.0)
l = 0.1
optimizer = Adam(learning_rate=1e-2)
training_variables = [w, e]
for i in range(n_iters):
    with tf.GradientTape() as t:
        model = tfd.JointDistributionNamedAutoBatched({
            "w": tfd.Independent(
                tfd.Laplace(loc=[0, 0], scale=l),
                reinterpreted_batch_ndims=1,
            ),
            "y": lambda w: tfd.Normal(loc=X @ tf.reshape(w, (-1, 1)), scale=e)
            })
        nll = -tf.reduce_sum(model.log_prob(w = w, y = y))
        grads = t.gradient(nll, training_variables)
        optimizer.apply_gradients(zip(grads, training_variables))

We find the following results:

In [ ]:
display(Math(r"\mathbf{w}"))
display(w_real)
display(Math(r"\mathbf{w_{map}}"))
display(w)

In [ ]:
display(Math(r"E"))
display(e_real)
display(Math(r"\tilde{E}"))
display(e)

Let's visualize the results:

In [ ]:
x_test = tf.cast(
        tf.reshape(tf.linspace(start=-1, stop=1, num=100), (-1, 1)),
        tf.float32
        )
X_test = tf.concat([x_test, tf.ones_like(x_test)], axis=1)
y_pred = X_test @ tf.reshape(w, (-1, 1))
y_pred_high = y_pred + 3 * tf.ones_like(y_pred) * e
y_pred_low = y_pred - 3 * tf.ones_like(y_pred) * e

In [ ]:
fig, ax = plt.subplots()
ax.scatter(tf.squeeze(x), tf.squeeze(y), alpha=0.2, label="data")
ax.plot(tf.squeeze(x_test), tf.squeeze(y_pred), label=r"$\tilde{y}$", color="k")
ax.fill_between(
    tf.squeeze(x_test),
    tf.squeeze(y_pred_low),
    tf.squeeze(y_pred_high),
    alpha=0.5, color="k"
)
ax.set_xlabel(r"$\mathbf{x}$")
ax.set_ylabel(r"$\mathbf{y}$")
ax.legend()
fig.show()

This result is equivalent to the Lasso regression model, which uses $L_1$ regularization (equivalent to the Laplace distribution). Nevertheless, it's possible to optimize any model by changing the distributions.

## Sampling From the Posterior
---

Up to this point, We've seen how to obtain the most likely parameters according to the posterior distribution (MAP estimation), however, it would be dessirable to have the posterior distribution or at least some samples from it.

This can be achieved through Markov Chain Monte Carlo (MCMC), to this end, let us define the following model:

$$
\begin{split}
P(\mathbf{y} | \mathbf{X}, \mathbf{w}, E) = N(\mathbf{y} = \mathbf{X} \cdot \mathbf{w}, \sigma = E)\\
P(\mathbf{w}) = N(\mathbf{w} = [0, 0], \Sigma = \mathbf{I})
\end{split}
$$

In [ ]:
model = tfd.JointDistributionNamedAutoBatched({
    "w": tfd.Normal(loc=tf.zeros(shape=(2, 1)), scale=4.0),
    "y": lambda w: tfd.Normal(loc=X @ w, scale=e_real)
    })

We can define the log function to optimize from this model:

In [ ]:
def log_prob(w):
    return model.log_prob(w=w, y=y)

Also, let us define the MCMC procedure as a `tensorflow` function:

In [ ]:
@tf.function
def mcmc():
    kernel = tfp.mcmc.NoUTurnSampler(
            target_log_prob_fn = log_prob,
            step_size=1e-3
            )
    return tfp.mcmc.sample_chain(
            num_results = 1000,
            num_burnin_steps = 500,
            current_state = [tf.zeros(shape=(2, 1))],
            kernel = kernel,
            trace_fn = lambda _, results: results.target_log_prob
            )

Now, we can compute samples from the posterior distribution $P(\mathbf{w}|\mathbf{y}, \mathbf{X}, E)$:

In [ ]:
samples, log_probs = mcmc()
w_posterior = tf.squeeze(samples[0])

We can visualize these distributions:

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.scatterplot(x = w_posterior[:, 0], y = w_posterior[:, 1], ax = ax)
sns.histplot(x = w_posterior[:, 0], y = w_posterior[:, 1], ax = ax)
sns.kdeplot(
    x = w_posterior[:, 0],
    y = w_posterior[:, 1],
    levels=5,
    color="w", linewidths=1,
    ax = ax
)
ax.set_xlabel(r"$\tilde{w}_1$")
ax.set_ylabel(r"$\tilde{w}_2$")
ax.set_title(r"$\mathbf{w} = " + f"[{w_real[0, 0]:.2f}, {w_real[1, 0]:.2f}]$")
fig.show()

## Approximating the Posterior
---

Another approach is to approximate the posterior distribution through variational inference.

In this case, we can use a surrogate posterior distribution $Q(\mathbf{w})$ to approximate $P(\mathbf{w} | \mathbf{y}, \mathbf{X}, E)$ through the minimization of the kullback-leibler divergence:

$$
KL(Q || P) = \int_{\mathbf{w}} Q(\mathbf{w}) \log \frac{Q(\mathbf{w})}{P(\mathbf{w} | \mathbf{y}, \mathbf{X}, E)} d\mathbf{w}
$$

From the Bayes rule, we obtain:

$$
KL(Q || P) = \int_{\mathbf{w}} Q(\mathbf{w}) \log{Q(\mathbf{w})} - \log{P(\mathbf{y}| \mathbf{w}, \mathbf{X}, E) P(\mathbf{w})} + \log(P(\mathbf{y}, \mathbf{X}, E)) d\mathbf{w}
$$

The term $P(\mathbf{y}, \mathbf{X}, E)$ does not depend on $\mathbf{w}$, therefore, the problem would be equivalent to minimizing the evidence lower bound function:

$$
ELBO(Q || P) = \underset{Q(\mathbf{w})}{\mathbb{E}}[\log Q(\mathbf{w}) - \log P(\mathbf{y}, \mathbf{w} | \mathbf{X}, E)]
$$

We can train a Bayesian linear regression with this approach, for instance, we can use the following surrogate posterior distribution:

$$
Q(\mathbf{w}) = N(\mathbf{w_{vi}}, \sigma_{vi})
$$

Therefore, we must learn its parameters:

In [ ]:
w_vi = tf.Variable([0., 0.])
sigma_vi = tf.Variable(1.0)

Let's define a the surrogate posterior distribution $Q(\mathbf{w})$ that we'll fit:

In [ ]:
surrogate_posterior = tfd.JointDistributionNamedAutoBatched({
    "w": tfd.Independent(tfd.Normal(loc=w_vi, scale=sigma_vi), reinterpreted_batch_ndims=1)
    })

Likewise, we can define the joint distribution $P(\mathbf{y}, \mathbf{w} | \mathbf{X}, E)$ according to the linear model that we want to learn:

In [ ]:
model = tfd.JointDistributionNamedAutoBatched({
    "w": tfd.Normal(loc=tf.zeros(shape=(2, )), scale=2.0),
    "y": lambda w: tfd.Normal(loc=X @ tf.reshape(w, (-1, 1)), scale=e_real)
    })

From this model, we can obtain the distribution for different $\mathbf{w}$ values, since $\mathbf{y}$ is constant:

In [ ]:
def log_prob(w):
    return model.log_prob(w=w, y=y)

Using these distributions, we can optimize the surrogate parameters using variational inference and gradient-based optimization:

In [ ]:
optimizer = Adam(learning_rate=1e-3)
loss = tfp.vi.fit_surrogate_posterior(
        target_log_prob_fn=log_prob,
        surrogate_posterior=surrogate_posterior,
        optimizer=optimizer,
        num_steps=5000,
        )

Let's see the learned parameters:

In [ ]:
display(Math(r"\mathbf{w}"))
display(w_real)
display(Math(r"\mathbf{w_{vi}}"))
display(w_vi)

Finally, we can visualize the surrogate posterior distribution:

In [ ]:
w1_range = np.linspace(0.9, 1.1, 100)
w2_range = np.linspace(-1.1, -0.9, 100)
W1, W2 = np.meshgrid(w1_range, w2_range)
W_grid = np.concatenate([W1.reshape(-1, 1), W2.reshape(-1, 1)], axis=1)
probs = surrogate_posterior.prob(w=W_grid).numpy().reshape(W1.shape)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.contourf(W1, W2, probs, cmap="gray")
ax.scatter([w_real[0]], [w_real[1]], label=r"$\mathbf{w}$")
ax.set_xlabel(r"$w_0$")
ax.set_ylabel(r"$w_1$")
ax.set_title(r"$Q(\mathbf{w})$")
ax.legend()
fig.show()